# Init

In [12]:
from IPython.display import display
from typing import List,Dict
import os
import sys
import psycopg2
from psycopg2.extras import NamedTupleCursor

os.chdir('/home/rchuzh/programming/image_labelling_shrdc')
from pathlib import Path
Path.cwd()
SRC = Path('/home/rchuzh/programming/image_labelling_shrdc/src')  # ROOT folder -> ./src
LIB_PATH = SRC / "lib"

if str(LIB_PATH) not in sys.path:
    sys.path.insert(0, str(LIB_PATH))  # ./lib
else:
    pass

In [13]:
from data_manager.database_manager import db_fetchone,db_no_fetch,db_fetchall
from core.utils.log import log_info,log_error
dsn = "host=localhost port=5432 dbname=eye user=shrdc password=shrdc"
def init_connection(dsn=None, connection_factory=None, cursor_factory=None, **kwargs):
    """ Connect to the PostgreSQL database server """

    try:
        # read connection parameters
        # params = config()

        # connect to the PostgreSQL server
        log_info('Connecting to the PostgreSQL database...')
        if kwargs:
            conn = psycopg2.connect(**kwargs)

        else:
            conn = psycopg2.connect(dsn, connection_factory, cursor_factory)

        # create a cursor
        with conn:
            with conn.cursor(cursor_factory=NamedTupleCursor) as cur:

                # execute a statement
                cur.execute('SELECT version();')
                conn.commit()

                # display the PostgreSQL database server version
                db_version = cur.fetchone().version
                log_info(f"PostgreSQL database version: {db_version}")
                log_info(f"PostgreSQL connection status: {conn.info.status}")
                log_info(
                    f"You are connected to database '{conn.info.dbname}' as user '{conn.info.user}' on host '{conn.info.host}' at port '{conn.info.port}'.")
        return conn
    except (Exception, psycopg2.DatabaseError) as error:
        log_error(error)
        conn = None


In [14]:
conn=init_connection(dsn)

2021-08-02 22:40:42.041 INFO    root: Connecting to the PostgreSQL database...
2021-08-02 22:40:42.048 INFO    root: PostgreSQL database version: PostgreSQL 13.3 (Ubuntu 13.3-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit
2021-08-02 22:40:42.049 INFO    root: PostgreSQL connection status: 0
2021-08-02 22:40:42.049 INFO    root: You are connected to database 'eye' as user 'shrdc' on host 'localhost' at port '5432'.


In [15]:
from typing import Union,List,Dict,Optional
import pandas as pd
def create_dataframe(data: Union[List, Dict, pd.Series], column_names: List = None, sort: bool = False, sort_by: Optional[str] = None, asc:bool=True,date_time_format: bool = False) -> pd.DataFrame:
    if data:

        df = pd.DataFrame(data, columns=column_names)
        df.index.name = ('No.')
        if date_time_format:
            df['Date/Time'] = pd.to_datetime(df['Date/Time'],
                                             format='%Y-%m-%d %H:%M:%S')

            # df.sort_values(by=['Date/Time'], inplace=True,
            #                ascending=False, ignore_index=True)
        if sort:

            df.sort_values(by=[sort_by], inplace=True,
                           ascending=asc, ignore_index=True)

            # dfStyler = df.style.set_properties(**{'text-align': 'center'})
            # dfStyler.set_table_styles(
            #     [dict(selector='th', props=[('text-align', 'center')])])

        return df

# 2/8/2021

In [5]:
query_sql="""
SELECT
            id AS "ID",
            name AS "Name",
            dataset_size AS "Dataset Size",
            updated_at AS "Date/Time",
            description AS "Description",
            dataset_path AS "Dataset Path",
            (SELECT dt.name AS "Deployment Type" from public.deployment_type dt where dt.id = d.deployment_id)
        FROM
            public.dataset d;
"""

In [6]:
query,column_names=db_fetchall(query_sql,conn,fetch_col_name=True)
query

[Record(ID=2, Name='My Second Dataset', Dataset_Size=4, Date_Time=datetime.datetime(2021, 7, 20, 14, 35, 18, 643911, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None)), Description=None, Dataset_Path='/home/rchuzh/.local/share/integrated-vision-inspection-system/app_media/dataset/My-Second-Dataset', Deployment_Type='Object Detection with Bounding Boxes'),
 Record(ID=3, Name='My Third Dataset', Dataset_Size=4, Date_Time=datetime.datetime(2021, 7, 20, 14, 38, 58, 453171, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None)), Description=None, Dataset_Path='/home/rchuzh/.local/share/integrated-vision-inspection-system/app_media/dataset/My-Third-Dataset', Deployment_Type='Object Detection with Bounding Boxes'),
 Record(ID=4, Name='My Fourth Dataset', Dataset_Size=4, Date_Time=datetime.datetime(2021, 7, 21, 20, 22, 29, 803351, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None)), Description='asdadsfads', Dataset_Path='/home/rchuzh/.local/share/integrated-visi

In [7]:
selection=[x.ID for x in query if x.Name=="My Third Dataset"][0]
selection

3

In [8]:
dataset_tmp = []
if query:
    for dataset in query:

        converted_datetime = dataset.Date_Time.strftime(
            '%Y-%m-%d %H:%M:%S')

        # convert datetime with TZ to (2021-07-30 12:12:12) format
        dataset = dataset._replace(
            Date_Time=converted_datetime)
        dataset_tmp.append(dataset)

In [9]:
df=create_dataframe(dataset_tmp,column_names=column_names,sort=True,sort_by='ID',asc=True,date_time_format=True)

In [11]:
df_loc=df.loc[:,"ID":"Date/Time"]
df_loc

,ID,Name,Dataset Size,Date/Time
0,2,My Second Dataset,4,2021-07-20 14:35:18
1,3,My Third Dataset,4,2021-07-20 14:38:58
2,4,My Fourth Dataset,4,2021-07-21 20:22:29
